In [1]:
import torch
import numpy as np
import random
import yaml
from argparse import Namespace
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from transformers import (
    AutoTokenizer, 
    AutoModelForMaskedLM
)
from tasks.dataset import FewShotDataset
from tasks.processors import compute_metrics_mapping

In [51]:
seed = 87

In [52]:
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [4]:
device = torch.device('cuda')
tokenizer = AutoTokenizer.from_pretrained('roberta-large')
generator = AutoModelForMaskedLM.from_pretrained('roberta-large').to(device)

In [45]:
with open("configs/config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
config = config['translation']['classification']
config = Namespace(**config)

In [46]:
task_name = "SST-2"
config.task_name = task_name.lower()
config.template = "*cls**sent_0*_It_was*mask*.*sep+*"
config.mapping = "{\'0\':\'terrible\',\'1\':\'great\'}"
config.data_dir = f"/home/c2hsieh/soft-Q-learning-for-text-generation/tasks/k-shot/{task_name}/16-{seed}"
config

Namespace(data_dir='/home/c2hsieh/soft-Q-learning-for-text-generation/tasks/k-shot/SST-2/16-100', debug_mode=False, demo_filter=False, demo_filter_model=None, demo_filter_rate=0.5, first_sent_limit=None, gpt3_in_context_head=False, gpt3_in_context_num=32, gpt3_in_context_tail=False, mapping="{'0':'terrible','1':'great'}", max_seq_length=512, num_sample=16, other_sent_limit=None, overwrite_cache=None, prompt=True, task_name='sst-2', template='*cls**sent_0*_It_was*mask*.*sep+*', template_list=None, use_demo=False)

In [53]:
task_name = "sst-5"
config.task_name = task_name.lower()
config.template = "*cls**sent_0*_It_was*mask*.*sep+*"
config.mapping = "{0:\'terrible\',1:\'bad\',2:\'okay\',3:\'good\',4:\'great\'}"
config.data_dir = f"/home/c2hsieh/soft-Q-learning-for-text-generation/tasks/k-shot/{task_name}/16-{seed}"
config

Namespace(data_dir='/home/c2hsieh/soft-Q-learning-for-text-generation/tasks/k-shot/sst-5/16-87', debug_mode=False, demo_filter=False, demo_filter_model=None, demo_filter_rate=0.5, first_sent_limit=None, gpt3_in_context_head=False, gpt3_in_context_num=32, gpt3_in_context_tail=False, mapping="{0:'terrible',1:'bad',2:'okay',3:'good',4:'great'}", max_seq_length=512, num_sample=16, other_sent_limit=None, overwrite_cache=None, prompt=True, task_name='sst-5', template='*cls**sent_0*_It_was*mask*.*sep+*', template_list=None, use_demo=False)

In [37]:
task_name = "MRPC"
config.task_name = task_name.lower()
config.template = "*cls**sent_0**mask*,*+sentl_1**sep+*"
config.mapping = "{'0':'No','1':'Yes'}"
config.data_dir = f"/home/c2hsieh/soft-Q-learning-for-text-generation/tasks/k-shot/{task_name}/16-{seed}"
config

Namespace(data_dir='/home/c2hsieh/soft-Q-learning-for-text-generation/tasks/k-shot/MRPC/16-100', debug_mode=False, demo_filter=False, demo_filter_model=None, demo_filter_rate=0.5, first_sent_limit=None, gpt3_in_context_head=False, gpt3_in_context_num=32, gpt3_in_context_tail=False, mapping="{'0':'No','1':'Yes'}", max_seq_length=512, num_sample=16, other_sent_limit=None, overwrite_cache=None, prompt=True, task_name='mrpc', template='*cls**sent_0**mask*,*+sentl_1**sep+*', template_list=None, use_demo=False)

In [54]:
prompt_100 = 'absolutely seriously Absolutely Simply Simply'
prompt_87 = 'absolutely unequivocallyliterally unequivocally unequivocally'
prompt_string = prompt_87
# prompt_string = tokenizer.convert_tokens_to_string(prompt.split())
prompt_string

'absolutely unequivocallyliterally unequivocally unequivocally'

In [55]:
dataset = FewShotDataset(config, tokenizer=tokenizer, mode="test")
dataset.use_learned_prompt = True
dataset.set_learned_prompt([prompt_string])
metrics_fn = compute_metrics_mapping[config.task_name]

In [56]:
[tokenizer.convert_ids_to_tokens(i) for i in dataset.get_labels_tok()]

['Ġterrible', 'Ġbad', 'Ġokay', 'Ġgood', 'Ġgreat']

In [57]:
dataloader = DataLoader(dataset, collate_fn=dataset.collate_fn, num_workers=4, pin_memory=True, batch_size=16, shuffle=False)
pred_labels, true_labels = [], []
for batch in tqdm(dataloader):
    with torch.no_grad():
        logits = generator(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device),
        ).logits.cpu()

    logits = logits[range(logits.shape[0]), batch['mask_pos'].squeeze()]
    pred_labels += logits[:, dataset.get_labels_tok()].argmax(1).tolist()
    true_labels += batch['labels'].squeeze().tolist()

metrics = metrics_fn(config.task_name, np.array(pred_labels), np.array(true_labels))
metrics

  0%|          | 0/139 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

{'acc': 0.3900452488687783}